# Music Generation with LSTM

In [1]:
import glob
import numpy as np
import pickle
from music21 import converter, instrument, stream, note, chord
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, Bidirectional, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K

In [2]:
def get_notes():
    """ Get all the notes and chords from the midi files in the ./full_set_beethoven_mozart directory. Call BEFORE train """
    notes = []
    durations = []
    files = "samples/*.mid"

    for file in glob.glob(files):
        midi = None
        try:
            midi = converter.parse(file)
        except:
            continue

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi) #Change to only grab the piano???
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch) + " " +  str(element.quarterLength))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder) + " " + str(element.quarterLength))
            elif isinstance(element, note.Rest):
                notes.append(str(element.name)  + " " + str(element.quarterLength))

    pickle.dump(notes, open('data/notes.p', 'wb'))

    return notes

In [3]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 16

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = to_categorical(network_output)

    return (network_input, network_output)

In [4]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    hidden_layers = 256
    dropout = 0.4
    
    """ Initializing model """
    model = Sequential()
    
    """ Adding LSTM Layers to Model """
    model.add(
        Bidirectional(
            LSTM(
                hidden_layers,
                dropout=dropout,
                return_sequences=True
            ),
            input_shape=(network_input.shape[1], network_input.shape[2])
        )
    )
    model.add(LayerNormalization())
    model.add(
        Bidirectional(
            LSTM(
                hidden_layers,
                dropout=dropout,
                return_sequences=True
            )
        )
    )
    model.add(LayerNormalization())
    model.add(
        Bidirectional(
            LSTM(
                hidden_layers,
                dropout=dropout
            )
        )
    )
    model.add(LayerNormalization())
    
    """ Add other layers after LSTM """
    model.add(Dropout(dropout))
    model.add(Dense(hidden_layers))
    model.add(Activation('relu'))
    model.add(LayerNormalization())
    model.add(Dropout(dropout))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer=Adam(clipnorm=1.0))

    return model

In [11]:
def train_network(load_notes=False, to_load_model=False, learning_rate=None):
    """ Train a Neural Network to generate music """
    notes = None
    model = None

    if load_notes:
        notes = pickle.load(open('data/notes.p', 'rb'))
    else:
        notes = get_notes()
    
    n_vocab = len(set(notes))
    network_input, network_output = prepare_sequences(notes, n_vocab)

    if to_load_model:
        model = load_model('weights.hdf5')
    else:
        model = create_network(network_input, n_vocab)

    if learning_rate != None:
        K.set_value(model.optimizer.learning_rate, learning_rate)

    checkpoint = ModelCheckpoint(
        'weights.hdf5',
        monitor='loss',
        save_best_only=True,
        mode='min'
    )

    callbacks_list = [checkpoint]
    
    model.summary()

    model.fit(
        x=network_input,
        y=network_output,
        batch_size=1024,
        epochs=3000,
        callbacks=callbacks_list
    )

In [12]:
train_network(True, True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 16, 512)           528384    
_________________________________________________________________
layer_normalization (LayerNo (None, 16, 512)           1024      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 16, 512)           1574912   
_________________________________________________________________
layer_normalization_1 (Layer (None, 16, 512)           1024      
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               1574912   
_________________________________________________________________
layer_normalization_2 (Layer (None, 512)               1024      
_________________________________________________________________
dropout (Dropout)            (None, 512)               0

Epoch 74/3000
52/52 [==============================] - 6s 113ms/step - loss: 2.2146
Epoch 75/3000
52/52 [==============================] - 6s 114ms/step - loss: 2.2124
Epoch 76/3000
52/52 [==============================] - 6s 113ms/step - loss: 2.2018
Epoch 77/3000
52/52 [==============================] - 6s 115ms/step - loss: 2.2144
Epoch 78/3000
52/52 [==============================] - 6s 113ms/step - loss: 2.2084
Epoch 79/3000
52/52 [==============================] - 6s 116ms/step - loss: 2.1942
Epoch 80/3000
52/52 [==============================] - 6s 116ms/step - loss: 2.1898
Epoch 81/3000
52/52 [==============================] - 6s 114ms/step - loss: 2.1941
Epoch 82/3000
52/52 [==============================] - 6s 113ms/step - loss: 2.2021
Epoch 83/3000
52/52 [==============================] - 6s 113ms/step - loss: 2.1994
Epoch 84/3000
52/52 [==============================] - 6s 119ms/step - loss: 2.1889
Epoch 85/3000
52/52 [==============================] - 6s 113ms/step - loss:

52/52 [==============================] - 6s 115ms/step - loss: 2.0211
Epoch 171/3000
52/52 [==============================] - 6s 114ms/step - loss: 2.0191
Epoch 172/3000
52/52 [==============================] - 6s 113ms/step - loss: 2.0233
Epoch 173/3000
52/52 [==============================] - 6s 119ms/step - loss: 2.0073
Epoch 174/3000
52/52 [==============================] - 6s 117ms/step - loss: 1.9998
Epoch 175/3000
52/52 [==============================] - 6s 114ms/step - loss: 2.0114
Epoch 176/3000
52/52 [==============================] - 6s 114ms/step - loss: 2.0159
Epoch 177/3000
52/52 [==============================] - 6s 118ms/step - loss: 1.9958
Epoch 178/3000
52/52 [==============================] - 6s 115ms/step - loss: 2.0071
Epoch 179/3000
52/52 [==============================] - 6s 113ms/step - loss: 2.0013
Epoch 180/3000
52/52 [==============================] - 6s 116ms/step - loss: 1.9980
Epoch 181/3000
52/52 [==============================] - 6s 113ms/step - loss: 1.

52/52 [==============================] - 6s 113ms/step - loss: 1.8679
Epoch 267/3000
52/52 [==============================] - 6s 113ms/step - loss: 1.8699
Epoch 268/3000
52/52 [==============================] - 6s 118ms/step - loss: 1.8609
Epoch 269/3000
52/52 [==============================] - 6s 116ms/step - loss: 1.8513
Epoch 270/3000
52/52 [==============================] - 6s 113ms/step - loss: 1.8691
Epoch 271/3000
52/52 [==============================] - 6s 118ms/step - loss: 1.8481
Epoch 272/3000
52/52 [==============================] - 6s 113ms/step - loss: 1.8561
Epoch 273/3000
52/52 [==============================] - 6s 118ms/step - loss: 1.8431
Epoch 274/3000
52/52 [==============================] - 6s 113ms/step - loss: 1.8495
Epoch 275/3000
52/52 [==============================] - 6s 115ms/step - loss: 1.8503
Epoch 276/3000
52/52 [==============================] - 6s 118ms/step - loss: 1.8418
Epoch 277/3000
52/52 [==============================] - 6s 114ms/step - loss: 1.

52/52 [==============================] - 6s 113ms/step - loss: 1.7192
Epoch 363/3000
52/52 [==============================] - 6s 117ms/step - loss: 1.7058
Epoch 364/3000
52/52 [==============================] - 6s 115ms/step - loss: 1.7276
Epoch 365/3000
52/52 [==============================] - 6s 117ms/step - loss: 1.7018
Epoch 366/3000
52/52 [==============================] - 6s 114ms/step - loss: 1.7070
Epoch 367/3000
52/52 [==============================] - 6s 113ms/step - loss: 1.7135
Epoch 368/3000
52/52 [==============================] - 6s 115ms/step - loss: 1.7108
Epoch 369/3000
52/52 [==============================] - 6s 113ms/step - loss: 1.7182
Epoch 370/3000
52/52 [==============================] - 6s 114ms/step - loss: 1.7149
Epoch 371/3000
52/52 [==============================] - 6s 113ms/step - loss: 1.7134
Epoch 372/3000
52/52 [==============================] - 6s 119ms/step - loss: 1.7009
Epoch 373/3000
52/52 [==============================] - 6s 113ms/step - loss: 1.

...

52/52 [==============================] - 6s 115ms/step - loss: 0.6825
Epoch 2555/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.6803
Epoch 2556/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.6744
Epoch 2557/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.6937
Epoch 2558/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.6725
Epoch 2559/3000
52/52 [==============================] - 6s 117ms/step - loss: 0.6790
Epoch 2560/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.6800
Epoch 2561/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.6875
Epoch 2562/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.6826
Epoch 2563/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.6720
Epoch 2564/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.6790
Epoch 2565/3000
52/52 [==============================] - 6s 116ms/step

52/52 [==============================] - 6s 115ms/step - loss: 0.6744
Epoch 2650/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.6706
Epoch 2651/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.6667
Epoch 2652/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.6709
Epoch 2653/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.6540
Epoch 2654/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.6623
Epoch 2655/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.6655
Epoch 2656/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.6609
Epoch 2657/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.6632
Epoch 2658/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.6676
Epoch 2659/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.6596
Epoch 2660/3000
52/52 [==============================] - 6s 119ms/step

52/52 [==============================] - 6s 114ms/step - loss: 0.6447
Epoch 2745/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.6577
Epoch 2746/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.6491
Epoch 2747/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.6523
Epoch 2748/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.6503
Epoch 2749/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.6572
Epoch 2750/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.6509
Epoch 2751/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.6511
Epoch 2752/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.6545
Epoch 2753/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.6470
Epoch 2754/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.6548
Epoch 2755/3000
52/52 [==============================] - 6s 115ms/step

52/52 [==============================] - 6s 114ms/step - loss: 0.6383
Epoch 2840/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.6341
Epoch 2841/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.6307
Epoch 2842/3000
52/52 [==============================] - 6s 117ms/step - loss: 0.6250
Epoch 2843/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.6504
Epoch 2844/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.6264
Epoch 2845/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.6323
Epoch 2846/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.6293
Epoch 2847/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.6391
Epoch 2848/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.6302
Epoch 2849/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.6397
Epoch 2850/3000
52/52 [==============================] - 6s 116ms/step

52/52 [==============================] - 6s 114ms/step - loss: 0.6167
Epoch 2935/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.6322
Epoch 2936/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.6215
Epoch 2937/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.6261
Epoch 2938/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.6318
Epoch 2939/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.6271
Epoch 2940/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.6164
Epoch 2941/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.6211
Epoch 2942/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.6246
Epoch 2943/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.6117
Epoch 2944/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.6224
Epoch 2945/3000
52/52 [==============================] - 6s 113ms/step

In [36]:
def prepare_sequences_output(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 16
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

In [50]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.random.choice(prediction.shape[1], p=(prediction[0] / sum(prediction[0])))
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [ ]:
#From: https://stackoverflow.com/questions/1806278/convert-fraction-to-float
def convert_to_float(frac_str):
    try:
        return float(frac_str)
    except ValueError:
        num, denom = frac_str.split('/')
        try:
            leading, num = num.split(' ')
            whole = float(leading)
        except ValueError:
            whole = 0
        frac = float(num) / float(denom)
        return whole - frac if whole < 0 else whole + frac

def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        pattern = pattern.split()
        temp = pattern[0]
        duration = pattern[1]
        pattern = temp
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a rest
        elif('rest' in pattern):
            new_rest = note.Rest(pattern)
            new_rest.offset = offset
            new_rest.storedInstrument = instrument.Piano() #???
            output_notes.append(new_rest)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += convert_to_float(duration)

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')

In [41]:
def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    notes = pickle.load(open('data/notes.p', 'rb'))

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences_output(notes, pitchnames, n_vocab)
    model = load_model('weights.hdf5', custom_objects={'SeqSelfAttention': SeqSelfAttention})
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

In [51]:
generate()